In [1]:
import pandas as pd
df= pd.read_csv("train.csv").applymap(lambda x: x.lower() if type(x) == str else x)
dftest=pd.read_csv("test.csv").applymap(lambda x: x.lower() if type(x) == str else x)
prior = df.groupby('class').size().div(len(df)) #count()['class']/len(df)
likelihood = {}
testlist=[]
featurelist=[]
numeric=[]
bins = 3

###pre proccessing##
for col in df:
    if(type(df[col][0])!=str):
        numeric.append(col)                                    #numeric features
        df[col]=pd.qcut(df[col],int(bins),duplicates="drop")   #numeric discretization
        
    if "class" not in col:
        featurelist.append(col)                                                       #feature list except class                           
        likelihood[col] = df.groupby(['class', col]).size().div(len(df)).div(prior)   ## Creating Likelihhod##

p_yes=1
p_no=1
for row in dftest.iterrows():
    p_yes=1
    p_no=1
    for key,val in row[1].items():
        if "class" not in key:
            if val in likelihood[key]['yes'].index and val in likelihood[key]['no'].index:
                p_yes*=likelihood[key]['yes'].loc[val]
                p_no*=likelihood[key]['no'].loc[val]
    p_yes*=prior['yes']     
    p_no*=prior['no']
    if p_yes>p_no:
         testlist.append("yes")
    else:
         testlist.append("no")
    
col1={'test':testlist}
final=pd.DataFrame(col1)
final['train']=df['class']
result=final.groupby(["test", "train"]).size()
success=result['no']['no']+result['yes']['yes']
counttest=len(final)
successrate=(success/counttest)*100
print("Success percent is: %.2f%%" % successrate)

Success percent is: 60.77%
